In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Sequential, callbacks
from keras.regularizers import l1
import pickle
import nltk
from random import randint, choice
import warnings
from flask import Flask, render_template, request

In [2]:
%matplotlib inline
warnings.filterwarnings('ignore')
plt.style.use('seaborn')

In [3]:
# read data

df = pd.read_csv('enron_dataset.csv', sep = ';').drop('Unnamed: 0', axis = 1)
df.head()

,Target,Message
0,internal company policy,Message-ID: <32715630.1075840547840.JavaMail.e...
1,alliances / partnerships,Message-ID: <8721012.1075863428380.JavaMail.ev...
2,internal company operations,Message-ID: <8687721.1075852656109.JavaMail.ev...
3,internal company operations,Message-ID: <16562450.1075846168630.JavaMail.e...
4,internal projects -- progress and strategy,Message-ID: <955111.1075858690252.JavaMail.eva...


In [85]:
# define a function to extract the datetime from emails

def get_datetime(message):
    date = message.split('\n')[1][11:-12]
    return datetime.strptime(date, '%d %b %Y %H:%M:%S')

In [5]:
# define a function to extract the subject from emails

def get_subject(message):
    
    subject = message.split('\n')
    for s in subject:
        if s.startswith('Subject'):
            return s[9:]

In [6]:
# list to store elements(email heading) to be removed

element_to_drop = ['Message-ID:', 'Date:', 'From:', 'To:', 'Subject:', 'Cc:', 'Mime-Version:',
 'Content-Type:', 'Content-Transfer-Encoding:', 'Bcc:', 'X-From:', 'X-To:', 'X-cc:', 'X-bcc:',
 'X-Folder:', 'X-Origin:', 'X-FileName:', 'cc', '\t', '--', 'Sent', ' --']

In [7]:
# define a function to get the email content from message

def get_content(message):
    
    message = message.split('\n')
    message_copy = message
    message_copy = [ele for ele in message_copy if ele != '']
    
    # remove the unwanted elements in the message
    for m in message:
        for e in element_to_drop:
            if m.startswith(e):
                message_copy.remove(m)
    return message_copy

In [8]:
# define a dictionary to create key-value pairs of labels

labels = df.Target.unique()
labels_cat = np.arange(len(labels))
labels_dict = dict(zip(labels_cat, labels))
labels_dict

{0: 'internal company policy',
 1: 'alliances / partnerships',
 2: 'internal company operations',
 3: 'internal projects -- progress and strategy',
 4: 'regulations and regulators (includes price caps)',
 5: ' company image -- current',
 6: 'california energy crisis / california politics',
 7: 'meeting minutes',
 8: 'political influence / contributions / contacts',
 9: 'legal advice',
 10: 'talking points',
 11: 'company image -- changing / influencing',
 12: 'trip reports'}

In [9]:
# define a list of stopwords

stopword_list = nltk.corpus.stopwords.words('english')
extra_stopwords = ['fw', 're']
stopword_list.extend(extra_stopwords)

In [10]:
# define a function to get the content of emails

def content_cleaning(content):
    
    content = [c.strip() for c in content]
    # remove empty string elements until there is none
    try:
        for _ in range(len(content)):
            content.remove('')
    except:
        content = ' '.join([c for c in content])
    finally:
        # remove None and empty list
        if content in [None, []]:
            content = ''
        return content

In [11]:
# define a function to remove punctuation, numbers, stop words, and redundant space from text

def text_cleaning(text):
    
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    text = text.lower().translate(str.maketrans('', '', string.digits))
    text = ' '.join([item for item in text.split() if item not in stopword_list])
    text = text.strip()
    return text

In [12]:
# define a function to get senders of email

def get_sender(text):
    return text.split('\n')[2][6:] 

In [13]:
# define a function to perform data cleaning

def data_cleaning(df, train = True):
    
    df2 = df.copy()

    # extract datetime
    df2['Datetime'] = df2.Message.apply(get_datetime)
    df2['Year'] = df2.Datetime.dt.year
    df2['Month'] = df2.Datetime.dt.month
    df2['Day'] = df2.Datetime.dt.day
    
    # email subject extraction
    df2['Subject'] = df2.Message.apply(get_subject)
    # email content extraction
    df2['Content'] = df2.Message.apply(get_content)
    
    if train:
        df2['Labels'] = df2['Target'].replace(list(labels_dict.values()), list(labels_dict.keys()))
    
    # get cleaned email subject and content
    df2['Subject_cleaned'] = df2['Subject'].apply(text_cleaning)
    df2['Content_cleaned'] = df2['Content'].apply(content_cleaning)
    df2['Content_cleaned'] = df2['Content_cleaned'].apply(text_cleaning)
    
    # get email senders
    emails = df.Message.apply(get_sender)
    # one-hot encoding for email senders
    emails = pd.get_dummies(emails)
    
    return df2, emails

In [14]:
# define a function to create TF-IDF for text

def tfidf(df):
    
    vectorizer = TfidfVectorizer(ngram_range = (1, 4), max_features = 4500, use_idf = True)
    subject_tfidf = vectorizer.fit_transform(df.Subject_cleaned)
    content_tfidf = vectorizer.fit_transform(df.Content_cleaned)
    
    return vectorizer, subject_tfidf, content_tfidf

In [15]:
# define a function to get receivers of email

def get_receiver(message):
    
    l = []
    for m in message:
        if m.startswith('X-To'):
            l.append(m[6:])
            l = l[0].split(', ')
    
    return l

In [16]:
# add receivers of email to the dataset

df2 = df.copy()
df2['Receiver'] = df2.Message.apply(lambda x: x.split('\n'))
df2['Receiver'] = df2.Receiver.apply(get_receiver)
df2.Receiver.head()

0    [GILBERT-SMITH@mailman.enron.com, Gilbert-smit...
1                                ['vkaminski@aol.com']
2    [Lay, Kenneth </O=ENRON/OU=NA/CN=RECIPIENTS/CN...
3                                    [Michael Terraso]
4    ['sbishop@gibbs-bruns.com', 'jalexander@gibbs-...
Name: Receiver, dtype: object

In [17]:
# define a function to get a list of email receivers

def get_receiver_list(receiver):
    
    receiver_list = []
    for r in receiver:
        receiver_list.append(r)
    receiver_list = [person for r in receiver_list for person in r if person != '']
    receiver_list = list(set(receiver_list))
    return receiver_list

In [18]:
# define a function to make one-hot encoding of receivers

def one_hot_encoding_receiver(df, receiver_list, train = True):
    
    df[receiver_list] = 0
    if train:
        df.drop(['Target', 'Message'], axis = 1, inplace = True)
    
    # loop through each row to perform one-hot encoding
    for row_index, row in enumerate(df.Receiver):
        for col in row:
            if col in df.columns:
                df.loc[row_index, col] = 1   
    df = df[receiver_list]
    return df

In [19]:
# get a list of receivers

receiver_list = get_receiver_list(df2.Receiver)
receiver_list[:10]

['Robert C Williams',
 'Eric Gadd',
 '"Bob Escalante" <rescalante@riobravo-gm.com>',
 'Neil Hong',
 'Karen Denne',
 '<cfi1@tca-us.com>',
 '"\'Lynn Lednicky (E-mail)\'" <lale@dynegy.com>',
 'Tim Belden <Tim Belden/ENRON@enronXgate>',
 '"\'Frank DeRosa (E-mail)\'" <frank.derosa@gen.pge.com>',
 '<tcwillia@duke-energy.com>']

In [20]:
%%time

# data cleaning for the dataset
df2, emails = data_cleaning(df2)
vectorizer, subject_tfidf, content_tfidf = tfidf(df2)
receivers = one_hot_encoding_receiver(df2, receiver_list)

CPU times: user 21.5 s, sys: 1.06 s, total: 22.6 s
Wall time: 31.9 s


In [21]:
# define a function to combine essential columns

def combine(df2, emails, receivers, subject_tfidf, content_tfidf):
    
    subject_tfidf_df = pd.DataFrame(subject_tfidf.toarray())
    content_tfidf_df = pd.DataFrame(content_tfidf.toarray())
    time = df2[['Year', 'Month', 'Day']]
    return pd.concat([time, emails, receivers, subject_tfidf_df, content_tfidf_df], axis = 1)

In [22]:
# prepare data for model training

X = combine(df2, emails, receivers, subject_tfidf, content_tfidf)
y = df2.Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [23]:
# define a neural network model 

model = Sequential([
#     Dense(1500, activation = 'relu', activity_regularizer = l1(0.001)),
    Dense(2000, activation = 'relu'),
    Dropout(0.4),
#     Dense(500, activation = 'relu'),
    Dense(len(labels_dict), activation = 'softmax')
])

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', 
              metrics = ['accuracy'])

# define early stopping in case that accuracy doesn't imporve
my_callbacks = callbacks.EarlyStopping(monitor = 'accuracy', patience = 4)
model.fit(X_train, y_train, epochs = 100, callbacks = [my_callbacks])
result = model.evaluate(X_test, y_test)
result

Epoch 1/100
22/22 [==============================] - 8s 240ms/step - loss: 40.2497 - accuracy: 0.1252
Epoch 2/100
22/22 [==============================] - 4s 201ms/step - loss: 29.5930 - accuracy: 0.1337
Epoch 3/100
22/22 [==============================] - 4s 163ms/step - loss: 21.3508 - accuracy: 0.1593
Epoch 4/100
22/22 [==============================] - 3s 155ms/step - loss: 15.6208 - accuracy: 0.1636
Epoch 5/100
22/22 [==============================] - 4s 162ms/step - loss: 10.6521 - accuracy: 0.1821
Epoch 6/100
22/22 [==============================] - 4s 161ms/step - loss: 7.6093 - accuracy: 0.1949
Epoch 7/100
22/22 [==============================] - 3s 151ms/step - loss: 4.4673 - accuracy: 0.2603
Epoch 8/100
22/22 [==============================] - 3s 144ms/step - loss: 2.5493 - accuracy: 0.3300
Epoch 9/100
22/22 [==============================] - 3s 142ms/step - loss: 2.1971 - accuracy: 0.3585
Epoch 10/100
22/22 [==============================] - 3s 150ms/step - loss: 1.8130 - a

[2.0537939071655273, 0.34090909361839294]

In [24]:
# save the model

now = datetime.now().strftime('%Y-%m-%d %H_%M_%S')
filename = f'{round(result[1], 3)}_{now}.pkl'

with open(filename, 'wb') as file:
    pickle.dump(model, file)

INFO:tensorflow:Assets written to: ram://552bdd1c-31b4-4ec4-9107-30f10feaa7c8/assets


In [96]:
# load saved model

model_loaded = pickle.load(open('0.398_2022-01-20 11_13_08.pkl', 'rb'))
model_loaded.evaluate(X_test, y_test)

6/6 [==============================] - 1s 46ms/step - loss: 2.0649 - accuracy: 0.3864


[2.064882516860962, 0.3863636255264282]

In [26]:
# get the configurations of the model

model_loaded.get_config()

{'name': 'sequential_15',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 10972),
    'dtype': 'float64',
    'sparse': False,
    'ragged': False,
    'name': 'dense_30_input'}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_30',
    'trainable': True,
    'dtype': 'float32',
    'units': 2000,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dropout',
   'config': {'name': 'dropout_11',
    'trainable': True,
    'dtype': 'float32',
    'rate': 0.4,
    'noise_shape': None,
    'seed': None}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_31',
    'trainable': True,
    'dtype': 'float32',
    'units': 13,
   

In [27]:
print(f"Input shape: {model_loaded.get_config()['layers'][1]['config']['units']}")
# print(f"L1: {model_loaded.get_config()['layers'][1]['config']['activity_regularizer']['config']['l1']}")
print(f"Dropout: {model_loaded.get_config()['layers'][2]['config']['rate']}")
print(f"Activation: {model_loaded.get_config()['layers'][3]['config']['activation']}")

Input shape: 2000
Dropout: 0.4
Activation: sigmoid


In [28]:
# define a function to clean and prepare user input message from web application

def pred_data_cleaning(message, receiver_list):
    message_copy = pd.DataFrame([message], columns = ['Message'])
    message_df, message_emails = data_cleaning(message_copy, train = False)
    message_df['Receiver'] = message_df.Message.apply(lambda x: x.split('\n'))
    message_df.drop(['Message', 'Datetime', 'Subject', 'Content'], axis = 1, inplace = True)
    
    for c in emails.columns:
        if c not in message_emails.columns:
            message_emails[c] = 0
    
    message_subject_tfidf = vectorizer.transform(message_df.Subject_cleaned)
    message_content_tfidf = vectorizer.transform(message_df.Content_cleaned)
    message_df['Receiver'] = message_df.Receiver.apply(get_receiver)
    receivers = one_hot_encoding_receiver(message_df, receiver_list, train = False)
    
    message_df_final = combine(message_df, message_emails, receivers, 
                               message_subject_tfidf, message_content_tfidf)

    return message_df_final

In [97]:
# test the predictions

num = choice(X_test.index)
m = df.Message[num]

message = pred_data_cleaning(m, receiver_list)
prediction = model_loaded.predict(message)
print(labels_dict[prediction.argmax()])
print(labels_dict[y_test[num]])

california energy crisis / california politics
political influence / contributions / contacts


In [66]:
testing = X_test.index
%store testing

Stored 'testing' (Int64Index)


In [100]:
# define a flask web application for user message input

app = Flask(__name__)

@app.route('/', methods = ['GET', 'POST'])
def home():
    if request.method == 'POST':
        user_message = request.form['text']
        user_message = user_message.replace('\\n', '\n')
        input_message = pred_data_cleaning(user_message, receiver_list)
        user_prediction = model_loaded.predict(input_message)
        pred_label = labels_dict[user_prediction.argmax()]
        return pred_label
    return render_template('index.html')

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Jan/2022 15:45:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 15:46:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 15:46:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 15:46:09] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 15:46:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 15:46:26] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 15:46:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 15:46:41] "POST / HTTP/1.1" 200 -
